In [ ]:
%pip install --user numpy
%pip install --user matplotlib

In [3]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [237]:
class MultiLayerPerceptronClassifier:
    
    def __init__(self, hidden_layer, suppress_output = False):
        self.W = None
        self.b = None
        self.suppress_output = False
        self.hidden_layer = hidden_layer
        
    def fit(self, X, y):
        #Se hará la suposición que todas las filas de X tienen el mismo número de columnas.
        self.build_network(len(X[0]))
        
        self.feed_forward(X[math.floor(random.random()*len(X))])

    
    #input_length esero entero positivo mayor a 1 que representa la cantidad de atributos de cada
    #dato de entrada (sin incluir el que se busca predecir).
    #hidden_dimensions es una lista que contiene las dimensiones de las capas escondidas de la red
    #si hidden_dimensions es [3,3,3] se crearán tres capas escondidas de 
    def build_network(self, input_length):
        
        dimensions = self.hidden_layer                   
        
        #W matriz de pesos. 3D, primera dimensión capa, Segunda dimensión neurona, tercera dimensión pesos. 
        #Entonces, W[0][0] contendría el arreglo con los pesos de la neurona 0 de la capa 0.
        W = []  
        for i in range(len(dimensions)): 
            W.append([])
        
        #b es un arreglo 2D que contiene el bias de cada neurona b[0][0] contiene el bias de la neurona 0 
        #de la capa 0
        b = []
        for i in range(len(dimensions)): 
            b.append([])
        
        
        #todas las neuronas en la primera capa deben tener input_lenght pesos. De ahí en adelante, 
        #cada neurona en la capa j tendrá len(W[j-1]) pesos.
        layer_width = dimensions[0]
        
        for i in range(layer_width):
            W[0].append([])
            W[0][i] = [random.uniform(-1,1) for j in range(input_length)]
            b[0].append(0)
            b[0][i] = random.uniform(-1,1)
            W[0][i] = np.array([W[0][i]])

            
        #Inicialización aleatoria de pesos y bias entre -1 y 1
        for i in range(1, len(dimensions)):
            layer_width = dimensions[i]
            for j in range(layer_width):
                #se inicializan n valores aleatorios entre -1 y 1 donde n = len(W[i-1])
                W[i].append([])
                W[i][j] = [random.uniform(-1,1) for j in range(len(W[i-1]))]
                b[i].append(0)
                #se crea un bias aleatorio para la neurona
                b[i][j] = random.uniform(-1,1) 
            
                W[i][j] = np.array([W[i][j]])

            
        self.W = W
        self.b = b
        
        if not self.suppress_output:
            print("Network Built")
            print("W")
            print(W)
            print("b")
            print(b)
            print("--------------------------------------------------------------------")
            
    #X es una entrada
    #W matriz de pesos. 3D, primera dimensión capa, Segunda dimensión neurona, tercera dimensión pesos. 
    #Entonces, W[0][0] contendría el arreglo con los pesos de la neurona 0 de la capa 0.
    #l será el número de capas.
    #b es un arreglo 2D que contiene el bias de cada neurona b[0][0] contiene el bias de la neurona 0 
    #de la capa 0.
    #Algoritmo se basa en el algoritmo 6.3 de GoodFellow et al 
    #y en el algoritmo de la página 217 de Grus en Data Science from Scratch
    def feed_forward(self, x):
        
        l = len(self.hidden_layer)
        
        #Salidas de cada neurona. Primera dimensión capa, segunda neuronas
        #outputs[0][0] representa la entrada 0 de X. 
        #outputs[1][0] representa la salida 0 de la primera capa escondida.
        
        outputs = [np.array([x])]
        
        for layer in range(1,l):
            
            layer_out = []
            for neuron in range(len(self.W[layer])):
                #a es el resultado de las salidas de la capa anterior por los pesos de la capa actual.
                
                X = outputs[layer-1].transpose()
                W = self.W[layer-1][neuron]
                
                a = np.dot(W,X)[0][0] + self.b[layer][neuron]
                layer_out.append(self.relu(a))
                
            outputs.append(np.array([layer_out]))
        if not self.suppress_output:
            print("Feed_forward outputs")
            print(outputs)
            print("--------------------------------------------------------------------")
        
    def relu(self, z):
        return max(0,z)

In [196]:
data_matrix = np.loadtxt(open("./msd_genre_dataset/fixed_ds.csv", "r"), delimiter=",", skiprows=0)



print("Filas de la matriz: " + str(len(data_matrix)))
print("Columnas de la matriz: " + str(len(data_matrix[0])))

y = data_matrix[:,len(data_matrix[0])-1]
X = np.delete(data_matrix, len(data_matrix[0])-1,1)

y = y.astype(int)


print(X)
print(y)


#Se intenta estandarizar X para lograr mejor desempeño. Sin embargo, no parece funcionar.
X = MinMaxScaler().fit_transform(X)

#Los datos del set de datos están agrupados por género. Es decir, primero están todas las filas que corresponden
#a 1 y después todas las que corresponden a -1. Se hace un shuffle para que, más tarde, en cross-validation
#no se creen unos modelos que predigan únicamente una clase.
np.random.shuffle(data_matrix)


print("X shape" + str(X.shape))
print("y shape" + str(y.shape))

Filas de la matriz: 8330
Columnas de la matriz: 31
[[-18.996       89.147        1.         ... 233.20616681 261.85070337
  240.83417734]
 [-19.347      125.825        4.         ... 227.2751235  261.64357048
  332.35653566]
 [ -9.472      121.707        4.         ... 549.49321044 481.14904868
  442.66313626]
 ...
 [ -9.494       88.976        4.         ... 519.52773394 538.89585608
  313.77593362]
 [ -7.617       67.929        3.         ... 591.85304812 598.05409088
  443.89380682]
 [-11.774       85.176        3.         ... 659.32142175 531.85019809
  607.21596134]]
[ 1  1  1 ... -1 -1 -1]
X shape(8330, 30)
y shape(8330,)


In [238]:
mlp = MultiLayerPerceptronClassifier(hidden_layer = [3, 3, 3], suppress_output = False)

mlp.fit(X, y)

Network Built
W
[[array([[ 0.57075196,  0.76957501,  0.42510119,  0.97000288,  0.81562676,
         0.73050498, -0.10422317, -0.84408533, -0.180482  ,  0.69938432,
         0.44922255,  0.7999806 , -0.0734062 , -0.61440517, -0.01505681,
         0.6586175 , -0.59784216,  0.6304097 ,  0.78162167,  0.24346346,
         0.99501732, -0.68764204,  0.74414126,  0.49496031,  0.57518101,
         0.79232529,  0.37517192,  0.58595769, -0.39005496, -0.11123341]]), array([[ 0.50793491, -0.54388764,  0.79620523,  0.88818573,  0.92563353,
         0.82877511,  0.14852161,  0.21671926, -0.58305934,  0.30851563,
         0.7186067 ,  0.3943051 ,  0.89256863,  0.83628487,  0.29697131,
         0.22546677, -0.32064613, -0.29196833, -0.00284457, -0.50450305,
         0.47741917,  0.65430553, -0.14545805,  0.47855857, -0.30369666,
         0.73783188, -0.45692139,  0.49958545,  0.34570769,  0.26205528]]), array([[-0.50011934, -0.69943011,  0.94612901,  0.12701007, -0.13671293,
         0.02665407, -0.119

array([ 8, 12, 16])